In [2]:
import matplotlib.pyplot as plt
import monai
import torch
import numpy as np
import pandas as pd
from pathlib import Path
import os
import re
import glob
from collections import defaultdict
import pydicom
import nibabel as nib
from scipy.ndimage import zoom
from monai.bundle import load_bundle_config
from huggingface_hub import hf_hub_download
import cv2 #The import-call for cv2 is "pip install opencv-python" (not cv2)

### Segmentation of DICOM files

Must know:
- ID 207: Add a folder named "sax" in cine and move all subfolders (with the weird names) in there for consistency

Good to know:
- All missing IDs have a "-" in column "Folders (y/n)". So when you find out folder-order and first and last for the missing ones, change that to "y" and run the code again. 
- We should ask about 187 (no sax folder), for now it also has a "-" and is ignored.
- Otherwise scroll down for the function calls. I structured it this way so it's easy to make into a .py file, but notebook is easier for debugging etc.
- Check out the TODO tag below about the affine to convert it into Nifti format and wether that is necessary for input into Giulia's code.

In [3]:
def parse_filename(filename):
    """
    Extracts sliceloc and triggertime values from a filename.

    Parameters:
        filename (str): Filename containing 'sliceloc_{val}_triggertime_{val}'.

    Returns:
        tuple[float | None, float | None]: Parsed sliceloc and triggertime as floats, or (None, None) if not found.
    """
    match = re.search(r"sliceloc_([-\d.]+)_triggertime_([-\d.]+)", filename)
    if match:
        return float(match.group(1)), float(match.group(2))
    return None, None

In [4]:
def get_relevant_files_n(df, base_path):
    """
    Selects one relevant file per slice location for each patient based on ED frame and apex–base range.

    For each patient, searches {base_path}/{ID}/cine/sax/ (recursively) for files named 
    like '...sliceloc_{val}_triggertime_{val}'. Keeps only slices within the apex–base 
    range and selects the earliest (ED Slice == 0) or latest frame per slice.

    Parameters:
        df (pd.DataFrame): df_y (see above); DataFrame of "ED_slices_and_timepoints.csv", without series-substructure
        base_path (str): Root path containing the patient folders.

    Returns:
        dict[str, list[str]]: Mapping from patient ID to selected file paths.
    """
    patient_files = {}

    for _, row in df.iterrows():
        pid = str(row["ID"]).strip()
        try:
            ed_slice = int(row["ED frame"])
            apex = float(row["apex"])
            base = float(row["base"])
        except (ValueError, TypeError):
            # Skip malformed rows
            continue

        folder = os.path.join(base_path, pid, "cine", "sax")
        if not os.path.isdir(folder):
            print(f"Warning: folder not found for patient {pid}")
            continue

        files = [p for p in Path(folder).rglob("*") if p.is_file()]
        parsed = []

        # Parse filenames
        for f in files:
            fname = f.name
            sliceloc, triggertime = parse_filename(fname)
            if sliceloc is not None and triggertime is not None:
                parsed.append((f, sliceloc, triggertime))

        if not parsed:
            print(f"Warning: no valid files for patient {pid}")
            continue

        # Group by sliceloc → list of triggertimes
        sliceloc_map = defaultdict(list)
        for f, sliceloc, triggertime in parsed:
            sliceloc_map[sliceloc].append((f, triggertime))

        lower, upper = sorted([apex, base])
        selected = []

        for sliceloc, items in sliceloc_map.items():
            if lower <= sliceloc <= upper:
                times = [tt for _, tt in items]
                if ed_slice == 0:
                    target_tt = min(times)
                else:
                    target_tt = max(times)

                # Add the file with this sliceloc + target triggertime
                for f, tt in items:
                    if tt == target_tt:
                        selected.append((sliceloc, str(f)))
                        break  # Only one per sliceloc
        
        # Sort by sliceloc before storing
        selected.sort(key=lambda x: x[0])
        patient_files[pid] = [f for _, f in selected]

        

    return patient_files



In [5]:
def get_relevant_files_y(df, base_path):
    """
    Selects one relevant file per folder-defined slice for each patient using ED frame.

    For each patient, looks inside {base_path}/{ID}/cine/sax/series_{folder}/ for files.
    The slice locations are inferred from subfolder names (e.g. 'series_25').
    The column 'folder order' lists all available series (in order),
    while 'apex' and 'base' define the first and last folder to include.

    Parameters:
        df (pd.DataFrame): DataFrame with columns 'ID', 'ED Slice', 'apex', 'base', and 'folder order'.
        base_path (str): Root path containing the patient folders.

    Returns:
        dict[str, list[str]]: Mapping from patient ID to selected file paths.
    """

    patient_files = {}

    for _, row in df.iterrows():
        pid = str(row["ID"]).strip()
        try:
            ed_slice = int(row["ED frame"])
            apex = int(row["apex"])
            base = int(row["base"])
            folder_order = str(row["folder order"]).strip()
        except (ValueError, TypeError):
            print(f"Could not extract values for ID {pid}")
            continue

        if not folder_order or folder_order.lower() == "nan":
            continue

        sax_root = os.path.join(base_path, pid, "cine", "sax")
        if not os.path.isdir(sax_root):
            print(f"Warning: folder not found for patient {pid}")
            continue

        # Get ordered folder list (as ints)
        order = [int(x) for x in folder_order.split("-") if x.isdigit()]
        lower_idx, upper_idx = order.index(apex), order.index(base)
        selected_series = order[lower_idx:upper_idx+1]

        selected = []

        for sliceloc in selected_series:
            series_path = os.path.join(sax_root, f"series_{sliceloc}")
            if not os.path.isdir(series_path):
                print(f"Warning: missing folder series_{sliceloc} for patient {pid}")
                continue

            files = glob.glob(os.path.join(series_path, "*"))
            triggertimes = []

            for f in files:
                _, tt = parse_filename(os.path.basename(f))
                if tt is not None:
                    triggertimes.append((f, tt))

            if not triggertimes:
                continue

            if ed_slice == 0:
                chosen_file = min(triggertimes, key=lambda x: x[1])[0]
            else:
                chosen_file = max(triggertimes, key=lambda x: x[1])[0]

            selected.append(chosen_file)

        patient_files[pid] = selected

    return patient_files


In [6]:
def is_excluded_size(f, pid=None):
    """
    Helper Function: Determines whether a file should be excluded based on its size.
    Applies a default size exclusion range (approx. 168 KB), but allows 
    per-patient overrides for specific cases where the real series has 
    a different file size.

    Parameters:
        f (Path): Path object pointing to the DICOM file.
        pid (str or int, optional): Patient ID used to apply override exclusion rules.

    Returns:
        bool: True if the file should be excluded based on its size, False otherwise.
    """
    # Default range: files that show as 168 KB
    default_range = (167000, 169000)

    # Manual overrides for specific PIDs
    overrides = {
        "4": (102000, 104000),     # PID 4 → exclude ~103 KB
        "169": (132000, 134000),   # PID 169 → exclude ~133 KB
    }

    lower, upper = overrides.get(str(pid), default_range)

    try:
        size = f.stat().st_size
        return lower <= size <= upper
    except FileNotFoundError:
        return True

In [14]:
def frame_number(fpath):
    match = re.search(r'frame(\d+)\.dcm', fpath.name)
    return int(match.group(1)) if match else -1

In [ ]:
def get_relevant_files_new(df, base_path):
    """
    Selects the ED frame DICOM file for each slice between apex and base for each patient.

    Parameters:
        df (pd.DataFrame): DataFrame with columns ['ID', 'ED frame', 'apex', 'base']
        base_path (str): Root directory containing patient folders (e.g., 'MAD_OUS_sorted')

    Returns:
        dict[str, list[str]]: Mapping from patient ID to selected file paths.
    """
    patient_files = {}

    for _, row in df.iterrows():
        try:
            pid = str(row["ID"]).strip()
            ed_frame = int(row["ED frame"])
            apex = int(row["apex"])
            base = int(row["base"])
        except (ValueError, TypeError):
            print("There was an error in reading the csv file.")
            continue 

        lower, upper = sorted([apex, base])
        selected = []

        for slice_num in range(lower, upper + 1):
            slice_folder = Path(base_path) / f"MAD_{pid}" / "Study01" / "cine" / "SAX" / f"slice_{slice_num}"
            if not slice_folder.exists():
                print(f"Folder slice_{slice_num} for patient {pid} can not be found and is skipped.")
                continue

            files = sorted(slice_folder.glob("frame*.dcm"), key=frame_number)
            if not files:
                print(f"Folder slice_{slice_num} for patient {pid} does not contain any frame*.dcm files.")
                continue

            # Filter out files of unwanted size
            valid_files = [f for f in files if not is_excluded_size(f, pid)]
            #print(valid_files[ed_frame].stat().st_size)

            if not valid_files:
                print(f"skipping patient {pid} and slice {slice_num} because all files are not valid (low quality added series)")
                continue  # all were 168 KB — skip this slice silently

            # Use reindexed list to get ED frame
            if ed_frame < len(valid_files):
                selected.append(str(valid_files[ed_frame]))
            else:
                print(f"ED_Frame {ed_frame} number larger than number of files in folder, Patient {pid}, Slice {slice_num} skipped")

            
        patient_files[pid] = selected

    return patient_files


In [16]:
def resize_image(img, target_shape):
    """
    Resize a 2D image to the target shape using OpenCV.

    Parameters:
        img (np.ndarray): Input image to resize.
        target_shape (tuple[int, int]): Desired output shape (height, width).

    Returns:
        np.ndarray: Resized image.
    """
    # Add padding if one of the image dimensions are smaller than the target shape
    if img.shape[0] < target_shape[0] or img.shape[1] < target_shape[1]:

        h, w = img.shape

        if h < target_shape[0]:
            pad_top = (target_shape[0] - h) // 2
            pad_bottom = target_shape[0] - h - pad_top
        else:
            pad_top = 0
            pad_bottom = 0


        if w < target_shape[1]:
            pad_left = (target_shape[1] - w) // 2
            pad_right = target_shape[1] - w - pad_left
        else:
            pad_left = 0
            pad_right = 0


        img = cv2.copyMakeBorder(
                        img,
                        top=pad_top,
                        bottom=pad_bottom,
                        left=pad_left,
                        right=pad_right,
                        borderType=cv2.BORDER_CONSTANT,
                        value=0  # or use the image mean
                    )
    
    # Resize the image if it exceeds the target shape
    if img.shape[0] > target_shape[0] or img.shape[1] > target_shape[1]:
        img = cv2.resize(img, (target_shape[1], target_shape[0]), interpolation=cv2.INTER_LINEAR)

    return img

In [17]:
def run_segmentation(files_dicts, output_root, save_as_stack: bool):
    """
    Runs MONAI ventricular segmentation on all DICOM files provided in files_dicts.

    Parameters:
        files_dicts (list[dict]): List of dicts (e.g., [files_n, files_y]) with {pid: [file_paths]}.
        output_root (str): Root folder where output NIfTI files will be saved.
        save_as_stack (bool): If True, saves all slices of a patient as a single NIfTI stack.
    """
    # Load MONAI network config & weights
    parser = load_bundle_config("MONAI", "train.json")
    net = parser.get_parsed_content("network_def")

    model_path = hf_hub_download(
        repo_id="MONAI/ventricular_short_axis_3label",
        filename="models/model.pt"
    )
    net.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))
    net.eval()

    target_shape = (256, 256) 

    for files_dict in files_dicts:
        for pid, paths in files_dict.items():
            num_slices = len(paths)
            img_stack = []
            seg_stack = []
            for idx, path in enumerate(paths):
                # Read and preprocess DICOM
                ds = pydicom.dcmread(path)
                img = ds.pixel_array.astype(np.float32)

                im_resized = resize_image(img, target_shape)

                # Normalize and add batch & channel dims
                normed_im = im_resized / im_resized.max()
                input_tensor = torch.from_numpy(normed_im).float()[None, None, :, :]

                # Predict
                with torch.no_grad():
                    pred = net(input_tensor)
                    pred = torch.softmax(pred[0], dim=0)
                    seg = torch.argmax(pred, dim=0).numpy()

                if save_as_stack:
                    img_stack.append(im_resized)
                    seg_stack.append(seg)
                else:
                    # Save
                    pid_folder = os.path.join(output_root, str(pid))
                    os.makedirs(pid_folder, exist_ok=True)

                    affine = np.eye(4)  # identity affine 
                    #TODO: Look into this. This is an identity affine to map from numpy array to nifti file format, but we should probably
                    # use the one from the DICOM - or does this not matter for input into Giulia's code?

                    nib.save(nib.Nifti1Image(im_resized, affine), os.path.join(pid_folder, f"{idx}_img.nii.gz"))
                    nib.save(nib.Nifti1Image(seg.astype(np.uint8), affine), os.path.join(pid_folder, f"{idx}_seg.nii.gz"))

                    print(f"Saved {pid} slice {idx}")
        
            if save_as_stack and num_slices > 0: # NOTE: The order of the slices for patients with no folder structure is not necessarily correct.
                
                # Save the entire stack as a single NIfTI file
                pid_folder = os.path.join(output_root, str(pid))
                os.makedirs(pid_folder, exist_ok=True)

                img_stack = np.stack(img_stack, axis=-1)
                seg_stack = np.stack(seg_stack, axis=-1)

                affine = np.eye(4)  # identity affine for the stack

                nib.save(nib.Nifti1Image(img_stack, affine), os.path.join(pid_folder, "img_stack.nii.gz"))
                nib.save(nib.Nifti1Image(seg_stack.astype(np.uint8), affine), os.path.join(pid_folder, "seg_stack.nii.gz"))

                print(f"ID {pid}: Saved image and segmentation stacks")
                

In [18]:
# This would be main in .py

# read in csv split on folders y/n
csv_file = "ED_slices_and_timepoints.csv" #For the future, once we structure our folders/files better we need to (probably) adjust this import
csv_file_new = "newDataSegmentation.csv" 
df = pd.read_csv(csv_file)
df_new = pd.read_csv(csv_file_new)
#display(df)

df.columns = df.columns.str.strip()
df_new.columns = df_new.columns.str.strip()
df["Folders (y/n)"] = df["Folders (y/n)"].str.strip().str.lower()

df_y = df[df["Folders (y/n)"] == 'y'].reset_index(drop=True)
df_n = df[df["Folders (y/n)"] == 'n'].reset_index(drop=True)

#display(df_n)
#display(df_y)

# Change this based on where you store the data
#base_path = "/Users/inad001/Documents/SSCP25/Data and scripts SSCP25/CMR_image_data/new data-dicom" #"/Users/inad001/Documents/SSCP25/Data and scripts SSCP25/CMR_image_data/new data-dicom"
base_path = "/Users/au698484/Documents/SSCP25_data/Data and scripts SSCP25/CMR_image_data/new data-dicom"
base_path_new = "/Users/au698484/Documents/SSCP25_data/newData/MAD_OUS_sorted"


files_n = get_relevant_files_n(df_n, base_path)
files_y = get_relevant_files_y(df_y, base_path)
files_new = get_relevant_files_new(df_new, base_path_new)

# Call segmentation and save segmentations and images under specified output file (change to match your own destination)
run_segmentation([files_n, files_y, files_new], output_root="/Users/au698484/Documents/SSCP25_data/segmented_data", save_as_stack=True) #"/Users/inad001/Documents/SSCP25/segmented_data"


[PosixPath('/Users/au698484/Documents/SSCP25_data/newData/MAD_OUS_sorted/MAD_1/Study01/cine/SAX/slice_3/frame0.dcm'), PosixPath('/Users/au698484/Documents/SSCP25_data/newData/MAD_OUS_sorted/MAD_1/Study01/cine/SAX/slice_3/frame1.dcm'), PosixPath('/Users/au698484/Documents/SSCP25_data/newData/MAD_OUS_sorted/MAD_1/Study01/cine/SAX/slice_3/frame2.dcm'), PosixPath('/Users/au698484/Documents/SSCP25_data/newData/MAD_OUS_sorted/MAD_1/Study01/cine/SAX/slice_3/frame3.dcm'), PosixPath('/Users/au698484/Documents/SSCP25_data/newData/MAD_OUS_sorted/MAD_1/Study01/cine/SAX/slice_3/frame4.dcm'), PosixPath('/Users/au698484/Documents/SSCP25_data/newData/MAD_OUS_sorted/MAD_1/Study01/cine/SAX/slice_3/frame5.dcm'), PosixPath('/Users/au698484/Documents/SSCP25_data/newData/MAD_OUS_sorted/MAD_1/Study01/cine/SAX/slice_3/frame6.dcm'), PosixPath('/Users/au698484/Documents/SSCP25_data/newData/MAD_OUS_sorted/MAD_1/Study01/cine/SAX/slice_3/frame7.dcm'), PosixPath('/Users/au698484/Documents/SSCP25_data/newData/MAD_OU

/opt/homebrew/Caskroom/miniconda/base/lib/python3.13/site-packages/monai/bundle/reference_resolver.py:226: UserWarning: Detected deprecated name 'optional_packages_version' in configuration file, replacing with 'required_packages_version'.
  warnings.warn(


ID 15: Saved image and segmentation stacks
ID 114: Saved image and segmentation stacks
ID 126: Saved image and segmentation stacks
ID 130: Saved image and segmentation stacks
ID 138: Saved image and segmentation stacks
ID 163: Saved image and segmentation stacks
ID 207: Saved image and segmentation stacks
ID 229: Saved image and segmentation stacks
ID 304: Saved image and segmentation stacks
ID 11: Saved image and segmentation stacks
ID 168: Saved image and segmentation stacks
ID 173: Saved image and segmentation stacks
ID 190: Saved image and segmentation stacks
ID 195: Saved image and segmentation stacks
ID 198: Saved image and segmentation stacks
ID 202: Saved image and segmentation stacks
ID 203: Saved image and segmentation stacks
ID 204: Saved image and segmentation stacks
ID 205: Saved image and segmentation stacks
ID 210: Saved image and segmentation stacks
ID 218: Saved image and segmentation stacks
ID 232: Saved image and segmentation stacks
ID 302: Saved image and segmentati

### Create h5 files

In [ ]:
# import nibabel as nib
# import nrrd

# stack_file = os.path.abspath("/Users/inad001/Documents/SSCP25/segmented_data/182/seg_stack.nii.gz.nii.nrrd.gz")

# data, header = nrrd.read(stack_file)
# seg_nii = nib.Nifti1Image(data, np.eye(4))  # Use identity affine for simplicity

# nib.save(seg_nii, "/Users/inad001/Documents/SSCP25/segmented_data/182/seg_stack.nii.gz")  # Save as NIfTI

In [12]:
from myo_cavity import create_h5_file

patients = list(files_n.keys()) + list(files_y.keys()) + list(files_new.keys())
segmentation_base_path = "/Users/au698484/Documents/SSCP25_data/segmented_data"
ouput_path = "seg_files/"

create_h5_file(patients=patients, base_path=segmentation_base_path, output_dir=ouput_path)

Data saved to data.h5
Data saved to data.h5
Data saved to data.h5
Data saved to data.h5
Number of valid slices: 9 out of 12
Valid slices: [2, 3, 4, 5, 6, 7, 8, 9, 10]
Data saved to data.h5
Data saved to data.h5
Number of valid slices: 6 out of 10
Valid slices: [0, 1, 2, 6, 8, 9]
Data saved to data.h5
Data saved to data.h5
Data saved to data.h5
Data saved to data.h5
Data saved to data.h5
Data saved to data.h5
Data saved to data.h5
Data saved to data.h5
Data saved to data.h5
Data saved to data.h5
Data saved to data.h5
Data saved to data.h5
Data saved to data.h5
Data saved to data.h5
Data saved to data.h5
Data saved to data.h5
Data saved to data.h5
Data saved to data.h5
Data saved to data.h5
Data saved to data.h5
Data saved to data.h5
Data saved to data.h5
Data saved to data.h5
Data saved to data.h5
Data saved to data.h5
Data saved to data.h5
Number of valid slices: 7 out of 10
Valid slices: [0, 2, 4, 5, 6, 7, 9]
Data saved to data.h5
Data saved to data.h5
Data saved to data.h5
Data saved

In [ ]:
import h5py

# Open the file in read mode
with h5py.File('seg_files/11.h5', 'r') as file:
    # Print all root-level groups/datasets
    def print_structure(name, obj):
        print(name)

    file.visititems(print_structure)

    # Access a specific dataset (example)
    dataset = file['resolution']
    data = dataset[:]  # Convert to a NumPy array
    print(data)
    
    # Print unique values in the dataset
    unique_values = np.unique(data)
    print("Unique values in LVmask:", unique_values)

In [ ]:
with h5py.File("seg_files/130.h5", 'r') as h5_file:
    segg = h5_file['RV_mask'][:]
    imres = h5_file['resolution'][:]
num_slices = segg.shape[0]  # Assuming slices are along the third dimension
grid_size = int(np.ceil(np.sqrt(num_slices))) # Determine grid size

fig, axes = plt.subplots(grid_size, grid_size, figsize=(8, 8))

for i in range(num_slices):
    row = i // grid_size
    col = i % grid_size
    label_1_slice = segg[i, :, :]
    axes[row, col].imshow(label_1_slice, cmap='gray')
    axes[row, col].set_title(f"Slice {i+1}")
    axes[row, col].axis('off')
plt.show()

### Segmentation of NIfTI files

This part collects CMR NIfTI files for the patients located in the ED_segmentation_data folder and predicts a segmentation for each patient.

In [ ]:
def get_cmr_nifti_files(base_path):
    """
    Gets all CMR NIfTI files for all patients in the specified base path.

    Parameters:
        base_path (str): Root path containing the patient folders.

    Returns:
        dict[str, list[str]]: Mapping from patient ID to lists of file paths.
    """
    patient_files = {}

    for patient in os.listdir(base_path):
        pid = patient.strip()

        patient_path = os.path.join(base_path, pid)

        if not os.path.isdir(patient_path):
            print(f"Warning: {patient_path} is not a directory")
            continue

        files = [os.path.join(patient_path, f) for f in os.listdir(patient_path) if f.startswith("cmr") and f.endswith(".nii")]

        if not files:
            print(f"Warning: no valid files found for patient {pid}")
            continue

        patient_files[pid] = files
    
    return patient_files

In [ ]:
def run_segmentation_nifti(files_dicts, output_root, save_as_stack: bool):
    """
    Runs MONAI ventricular segmentation on all CMR NIfTI files provided in files_dicts.

    Parameters:
        files_dicts (list[dict]): List of dicts (e.g., [files_n, files_y]) with {pid: [file_paths]}.
        output_root (str): Root folder where output NIfTI files will be saved.
        save_as_stack (bool): If True, saves all slices of a patient as a single NIfTI stack.
    """
    # Load MONAI network config & weights
    parser = load_bundle_config("MONAI", "train.json")
    net = parser.get_parsed_content("network_def")

    model_path = hf_hub_download(
        repo_id="MONAI/ventricular_short_axis_3label",
        filename="models/model.pt"
    )
    net.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))
    net.eval()

    target_shape = (256, 256) 

    for files_dict in files_dicts:
        for pid, paths in files_dict.items():
            # Read and preprocess NIfTI file
            img_stack = np.array(nib.load(paths[0]).get_fdata().astype(np.float32))

            num_slices = img_stack.shape[2]
            processed_img_stack = []
            seg_stack = []

            for idx in range(img_stack.shape[2]):
                img = img_stack[:, :, idx]  # Get the current slice

                im_resized = resize_image(img, target_shape)
                
                # Adjust contrast
                # im_resized = cv2.convertScaleAbs(im_resized, alpha=1.465, beta=0.0) # Used alpha-value provided by Giulia (this adjusts contrast)

                normed_im = im_resized / (np.max(im_resized)) 
                input_tensor = torch.from_numpy(normed_im).float()[None, None, :, :]

                # Predict
                with torch.no_grad():
                    pred = net(input_tensor)
                    pred = torch.softmax(pred[0], dim=0)
                    seg = torch.argmax(pred, dim=0).numpy()

                if save_as_stack:
                    processed_img_stack.append(im_resized)
                    seg_stack.append(seg)
                else:
                    # Save
                    pid_folder = os.path.join(output_root, str(pid))
                    os.makedirs(pid_folder, exist_ok=True)

                    affine = np.eye(4)  # identity affine 
                    #TODO: Look into this. This is an identity affine to map from numpy array to nifti file format, but we should probably
                    # use the one from the DICOM - or does this not matter for input into Giulia's code?

                    nib.save(nib.Nifti1Image(im_resized, affine), os.path.join(pid_folder, f"{idx}_img.nii.gz"))
                    nib.save(nib.Nifti1Image(seg.astype(np.uint8), affine), os.path.join(pid_folder, f"{idx}_seg.nii.gz"))

                    print(f"Saved {pid} slice {idx}")
        
            if save_as_stack and num_slices > 0: # NOTE: The order of the slices for patients with no folder structure is not necessarily correct.
                
                # Save the entire stack as a single NIfTI file
                pid_folder = os.path.join(output_root, str(pid))
                os.makedirs(pid_folder, exist_ok=True)

                processed_img_stack = np.stack(processed_img_stack, axis=-1)
                seg_stack = np.stack(seg_stack, axis=-1)

                affine = np.eye(4)  # identity affine for the stack

                nib.save(nib.Nifti1Image(processed_img_stack, affine), os.path.join(pid_folder, "img_stack.nii.gz"))
                nib.save(nib.Nifti1Image(seg_stack.astype(np.uint8), affine), os.path.join(pid_folder, "seg_stack.nii.gz"))

                print(f"ID {pid}: Saved image and segmentation stacks")

In [ ]:
# Change this based on where you store the data
base_path = os.path.abspath("/Users/inad001/Documents/SSCP25/Data and scripts SSCP25/ED_segmentation_data/segmentation_stacks")

cmr_files = get_cmr_nifti_files(base_path)

# Run segmentation on the CMR files
run_segmentation_nifti([cmr_files], output_root="/Users/inad001/Documents/SSCP25/segmented_nifti", save_as_stack=True)